In [220]:
import pandas as pd
import os
from os import path as osp

In [238]:
ROOT_DIR = osp.abspath('..')# change this
DATA_PATH = osp.join(ROOT_DIR, 'cel/data')

print(ROOT_DIR)
print(DATA_PATH)

/Users/uyen/simcel/simcel
/Users/uyen/simcel/simcel/cel/data


### Read raw data

In [252]:
raw_data_file = osp.abspath(osp.join(DATA_PATH, 'simcel-6pk70-1jk5iqdp-train_v9rqX0R.csv'))
df = pd.read_csv(raw_data_file)

df.sample(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
6193,FDX52,11.500,Regular,0.041995,Frozen Foods,194.7820,OUT035,2004,Small,Tier 2,Supermarket Type1,3475.4760
3363,NCK29,NaN,Low Fat,0.000000,Health and Hygiene,125.1730,OUT027,1985,Medium,Tier 3,Supermarket Type3,3572.0170
6738,NCJ43,6.635,Low Fat,0.027069,Household,174.9396,OUT046,1997,Small,Tier 1,Supermarket Type1,1744.3960
1533,NCG43,20.200,Low Fat,0.074390,Household,91.4462,OUT045,2002,NaN,Tier 2,Supermarket Type1,1943.4702
4846,FDV09,12.100,Low Fat,0.020565,Snack Foods,148.2734,OUT035,2004,Small,Tier 2,Supermarket Type1,3266.4148


In [253]:
print(f"Number of records is: {len(df)}")
print(f"Number of columns is: {len(df.columns)}")
print(f"Name of columns: {df.columns.tolist()}")

Number of records is: 8523
Number of columns is: 12
Name of columns: ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type', 'Item_Outlet_Sales']


In [254]:
# Statistics of some float fields:
df.describe(include=['float'])

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,2181.288914
std,4.643456,0.051598,62.275067,1706.499616
min,4.555000,0.000000,31.290000,33.290000
25%,8.773750,0.026989,93.826500,834.247400
50%,12.600000,0.053931,143.012800,1794.331000
75%,16.850000,0.094585,185.643700,3101.296400
max,21.350000,0.328391,266.888400,13086.964800


In [255]:
# Statistics of some int fields:
df.describe(include=['int'])

,Outlet_Establishment_Year
count,8523.000000
mean,1997.831867
std,8.371760
min,1985.000000
25%,1987.000000
50%,1999.000000
75%,2004.000000
max,2009.000000


In [256]:
# Statistics of some categorical fields:
df.describe(include=['object'])

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


### Preprocessing & ETL

In [257]:
attrs = df.columns.values # all attribute in raw data
categorized_attrs = ['Item_Fat_Content', 'Item_Type', 'Outlet_Establishment_Year', 'Outlet_Type'] # all categorized fields in raw data
outlet_attrs = ['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Type', 'Outlet_Location_Type']
item_attrs = ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Type'] # Item attributes 
cross_attrs = ['Item_MRP', 'Item_Visibility', 'Item_Outlet_Sales']

In [258]:
# Check if every attribute wrote in save format
for col in df.columns:
    if col in categorized_attrs:
        print(col, df[col].unique())

Item_Fat_Content ['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
Item_Type ['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']
Outlet_Establishment_Year [1999 2009 1998 1987 1985 2002 2007 1997 2004]
Outlet_Type ['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


In [259]:
# We can see some value is being written in different formats, for example: 'Low Fat', 'low fat', 'LF'
# then we now synchronize them: 
df.Item_Fat_Content.replace(to_replace='low fat', value='Low Fat', inplace=True)
df.Item_Fat_Content.replace(to_replace='LF', value='Low Fat', inplace=True)
df.Item_Fat_Content.replace(to_replace='reg', value='Regular', inplace=True)

#### Outlets

In [260]:
# Delete duplicates, fillin missing vals
df_outlets = df[outlet_attrs]
df_outlets = df_outlets.drop_duplicates(subset=['Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Type', 'Outlet_Location_Type'])

outlets_unique = df.Outlet_Identifier.unique()
assert len(outlets_unique) == len(df_outlets)

# After removing duplications, the num of unique vals == len of table, so there is no Nan value that can be replace
df_outlets.sample(5)

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Type,Outlet_Location_Type
1,OUT018,2009,Medium,Supermarket Type2,Tier 3
19,OUT035,2004,Small,Supermarket Type1,Tier 2
7,OUT027,1985,Medium,Supermarket Type3,Tier 3
23,OUT019,1985,Small,Grocery Store,Tier 1
9,OUT017,2007,NaN,Supermarket Type1,Tier 2


In [261]:
# Save outlets
outlets_data_path = osp.join(DATA_PATH, 'outlets.csv')
df_outlets.to_csv(outlets_data_path, index=None)

#### Items

In [263]:
# item_attrs = ['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Type'] # Item attributes 
df_items = df[item_attrs]
grouped_items = df_items.groupby('Item_Identifier').groups
_dfs = pd.DataFrame()

# Impute missing value
for i in grouped_items:
    _df = df_items.iloc[grouped_items[i]]
    _df = _df.sort_values(by='Item_Weight')
    _df.ffill(inplace=True)
    _dfs = pd.concat([_dfs, _df.head(1)])

_dfs.sample(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Type
2679,NCQ18,15.75,Low Fat,Household
2273,DRD27,18.75,Low Fat,Dairy
905,FDX16,17.85,Low Fat,Frozen Foods
3609,FDC14,14.50,Regular,Canned
464,DRI51,17.25,Low Fat,Dairy


In [264]:
# Save cleaned data
items_data_path = osp.join(DATA_PATH, 'items.csv')
_dfs.to_csv(items_data_path, index=None)

In [265]:
df.columns.values # all attribute in raw data

array(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content',
       'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'], dtype=object)

In [266]:
df_cleaned = df[['Item_Identifier', 'Outlet_Identifier', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']]
df_cleaned.sample(5)

,Item_Identifier,Outlet_Identifier,Item_Visibility,Item_MRP,Item_Outlet_Sales
3876,NCI06,OUT049,0.047792,180.7660,5033.4480
2891,DRM47,OUT035,0.043777,192.9846,6114.7072
7833,FDU34,OUT027,0.074831,125.9046,4482.1656
2974,FDP57,OUT018,0.052658,103.6990,1031.9900
7937,FDE34,OUT017,0.108502,183.6634,2362.9242


In [267]:
cleaned_data_path = osp.join(DATA_PATH, 'simcel.csv')
df_cleaned.to_csv(cleaned_data_path, index=None)

### Let's try stock data

In [204]:
import yfinance as yf

msft = yf.Ticker("nvda")
